In [1]:
import os
import json
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import random
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import os
import json
from typing import List, Tuple

# ==================================
# Config
# ==================================
# List of multiple camera roots (add/remove as needed)
FRAMES_ROOTS = [
    "./cam2-resized_frames",
    "./cam3-resized_frames",
]
LABELS_ROOT = "./micro_activity_output"
SEQUENCE_LENGTH = 16

# Keywords for class detection
FALL_KEYWORDS = ["fall_"]
LYING_KEYWORDS = ["lying_", "_lying"]


def safe_frame_index(filename: str) -> int:
    """
    Extract a numeric frame index from a filename like 'frame_00012.jpg'.
    """
    # Extract frame index safely (in case filenames differ, prevents crashes)
    try:
        core = os.path.splitext(filename)[0]
        tokens = core.split("_")
        # Search backwards for a numeric token
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        # Fallback: keep only digits
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1


def load_labels_for_activity(labels_root: str, activity_name: str) -> List[str]:
    """
    Load per-frame micro activity labels for a given activity.
    """
    # Labels are shared across cameras for the same activity
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return []

    with open(label_json_path, "r") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)

    # Sort labels by frame index (keys should be numeric)
    try:
        items = sorted(labels_dict.items(), key=lambda x: int(x[0]))
    except Exception:
        # If keys are not purely numeric, fall back to alphabetical order
        items = sorted(labels_dict.items(), key=lambda x: x[0])

    labels = [lbl for _, lbl in items]
    return labels


def sequence_majority_label(seq_labels: List[str], seq_len: int) -> int:
    """
    Decide the sequence label using majority voting.
    Returns: 0=ADL, 1=Fall, 2=Lie Down
    """
    # Count votes for fall and lying
    fall_votes = sum(lbl.startswith(tuple(FALL_KEYWORDS)) for lbl in seq_labels)
    lying_votes = sum(any(k in lbl for k in LYING_KEYWORDS) for lbl in seq_labels)

    if fall_votes >= (seq_len // 2):
        return 1  # Fall
    if lying_votes >= (seq_len // 2):
        return 2  # Lie Down
    return 0  # ADL


def generate_sequences_and_labels(
    frame_roots: List[str],
    labels_root: str,
    seq_len: int = 16
) -> List[Tuple[List[str], int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), build sequences and labels.
    Returns a list of tuples: (list_of_frame_paths, label_id)
    """
    # Iterate over multiple camera roots (no need to merge folders physically)
    data = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))  # for logging

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels = load_labels_for_activity(labels_root, activity_name)
            if not labels:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            # Each subfolder = one video of the same activity from this camera
            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                # Sort frames robustly by numeric index
                frame_files.sort(key=safe_frame_index)
                frame_paths = [os.path.join(frames_dir, f) for f in frame_files]

                # Safe alignment: trim to minimum length between frames and labels
                n = min(len(frame_paths), len(labels))
                if n < seq_len:
                    # Skip clips that are too short
                    continue

                use_paths = frame_paths[:n]
                use_labels = labels[:n]

                # Sliding window sequence generation
                num_sequences = (n - seq_len) + 1
                for i in range(num_sequences):
                    seq_paths = use_paths[i:i + seq_len]
                    seq_labels = use_labels[i:i + seq_len]
                    label = sequence_majority_label(seq_labels, seq_len)
                    data.append((seq_paths, label))

    return data


# =======================
# Run
# =======================
if __name__ == "__main__":
    dataset = generate_sequences_and_labels(FRAMES_ROOTS, LABELS_ROOT, SEQUENCE_LENGTH)
    print(f"Total sequences: {len(dataset)}")


[cam2-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam2-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_and_Bed_Rolling
[cam3-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
Total sequences: 702093


In [3]:
# Count the distribution
adl_count = sum(1 for _, label in dataset if label == 0)
fall_count = sum(1 for _, label in dataset if label == 1)
lying_count = sum(1 for _, label in dataset if label == 2)

print(f"Total sequences: {len(dataset)}")
print(f"ADL sequences: {adl_count}")
print(f"Fall sequences: {fall_count}")
print(f"Lie Down sequences: {lying_count}")

# Show example
print("\nExample sequence:")
for i in range(3):  # Show first 3 frames of first sample
    print(dataset[0][0][i])
print("Label (0=ADL,1=Fall,2=Lie Down):", dataset[0][1])

Total sequences: 702093
ADL sequences: 591614
Fall sequences: 43913
Lie Down sequences: 66566

Example sequence:
./cam2-resized_frames/Actor_1_Bed/Actor_1_Bed CAM 2/frame_0.jpg
./cam2-resized_frames/Actor_1_Bed/Actor_1_Bed CAM 2/frame_1.jpg
./cam2-resized_frames/Actor_1_Bed/Actor_1_Bed CAM 2/frame_2.jpg
Label (0=ADL,1=Fall,2=Lie Down): 0


In [4]:
import pickle

# Save the dataset
with open("./f_seq_r_3dcnn-convlstm2d-2cams.pkl", "wb") as f:
    pickle.dump(dataset, f)

print("Dataset saved successfully.")

Dataset saved successfully.


In [2]:
import os
import json
import pickle
from collections import Counter
from typing import List, Tuple

# ==================================
# Config
# ==================================
FRAMES_ROOTS = [
    "./cam2-resized_frames",
    "./cam3-resized_frames",
]
LABELS_ROOT = "./micro_activity_output"
SEQUENCE_LENGTH = 16
STRIDE = 8  

# Keywords for class detection
FALL_KEYWORDS = ["fall_"]
LYING_KEYWORDS = ["lying_", "_lying"]


def safe_frame_index(filename: str) -> int:
    """Extract a numeric frame index from a filename like 'frame_00012.jpg'."""
    try:
        core = os.path.splitext(filename)[0]
        tokens = core.split("_")
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1


def load_labels_for_activity(labels_root: str, activity_name: str) -> List[str]:
    """Load per-frame micro activity labels for a given activity."""
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return []
    with open(label_json_path, "r") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)
    try:
        items = sorted(labels_dict.items(), key=lambda x: int(x[0]))
    except Exception:
        items = sorted(labels_dict.items(), key=lambda x: x[0])
    labels = [lbl for _, lbl in items]
    return labels


def sequence_majority_label(seq_labels: List[str], seq_len: int) -> int:
    """
    Decide the sequence label using majority voting.
    Returns: 0=ADL, 1=Fall, 2=Lie Down
    """
    fall_votes = sum(lbl.startswith(tuple(FALL_KEYWORDS)) for lbl in seq_labels)
    lying_votes = sum(any(k in lbl for k in LYING_KEYWORDS) for lbl in seq_labels)
    if fall_votes >= (seq_len // 2):
        return 1  # Fall
    if lying_votes >= (seq_len // 2):
        return 2  # Lie Down
    return 0  # ADL


def generate_sequences_and_labels(
    frame_roots: List[str],
    labels_root: str,
    seq_len: int = 16,
    stride: int = 8,
) -> List[Tuple[List[str], int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), build sequences and labels.
    Returns a list of tuples: (list_of_frame_paths, label_id)
    """
    assert seq_len > 0, "seq_len must be > 0"
    assert stride > 0, "stride must be > 0"

    data: List[Tuple[List[str], int]] = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels = load_labels_for_activity(labels_root, activity_name)
            if not labels:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                frame_files.sort(key=safe_frame_index)
                frame_paths = [os.path.join(frames_dir, f) for f in frame_files]

                n = min(len(frame_paths), len(labels))
                if n < seq_len:
                    continue

                use_paths = frame_paths[:n]
                use_labels = labels[:n]

                # --- STRIDE-aware sliding window ---
                for i in range(0, n - seq_len + 1, stride):
                    seq_paths = use_paths[i:i + seq_len]
                    seq_labels = use_labels[i:i + seq_len]
                    label = sequence_majority_label(seq_labels, seq_len)
                    data.append((seq_paths, label))

    return data


# =======================
# Run inside Jupyter
# =======================
dataset = generate_sequences_and_labels(
    frame_roots=FRAMES_ROOTS,
    labels_root=LABELS_ROOT,
    seq_len=SEQUENCE_LENGTH,
    stride=STRIDE,
)

print(f"Total sequences: {len(dataset)}")
dist = Counter([y for _, y in dataset])
print("Class distribution (0=ADL, 1=Fall, 2=LieDown):", dict(dist))

# Optional: save pickle
with open(f"f_seq_stride{STRIDE}.pkl", "wb") as f:
    pickle.dump(dataset, f)
print(f"Saved pickle to f_seq_stride{STRIDE}.pkl")


[cam2-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam2-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_and_Bed_Rolling
[cam3-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
Total sequences: 87869
Class distribution (0=ADL, 1=Fall, 2=LieDown): {0: 74068, 2: 8315, 1: 5486}
Saved pickle to f_seq_stride8.pkl
